In [ ]:
from transformers import AutoTokenizer, BartForConditionalGeneration
import pandas as pd
import nltk
import math
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# summaries = pd.read_csv("/content/summaries.csv")
article_unique = pd.read_csv("/content/khan_article_unique.csv")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")

In [ ]:
def chunkify(text):
  sentences = nltk.tokenize.sent_tokenize(text)
  length = 0
  chunk = ""
  chunks = []
  count = -1
  num_chunks = math.ceil(len(tokenizer.tokenize(text))/tokenizer.max_len_single_sentence)
  tok_per_chunk = math.ceil(len(tokenizer.tokenize(text))/num_chunks)

  for sentence in sentences:
    count += 1
    combined_length = len(tokenizer.tokenize(sentence)) + length

    if combined_length <= tok_per_chunk:
      chunk += sentence + " "
      length = combined_length

    else:
      chunks.append(chunk.strip())
      length = 0
      chunk = ""
      chunk += sentence + " "
      length = len(tokenizer.tokenize(sentence))

  print(len(tokenizer.tokenize(text)))
  print(num_chunks)

  return chunks

In [ ]:
def summarize(chunks):
  summary = ""
  inputs = [tokenizer(chunk, return_tensors="pt") for chunk in chunks]
  max_length = round(tokenizer.max_len_single_sentence / len(inputs))
  min_length = round(max_length*(5/6))

  for i in range(len(inputs)):
    # print(inputs[i])
    inputs[i].to(device)
    output = model.generate(**inputs[i], min_length = min_length, max_length = max_length)
    part = tokenizer.decode(*output, skip_special_tokens = True)
    summary += part
  return summary

In [ ]:
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

In [ ]:
partial_summaries = pd.read_csv("/content/khan_summaries.csv")
summaries = list(partial_summaries.iloc[:,0][:len(partial_summaries)-1])
# summaries = []
articles = article_unique.iloc[:,0][len(summaries):]
for art in articles:
  if len(tokenizer.tokenize(art)) < tokenizer.max_len_single_sentence:
    print(len(tokenizer.tokenize(art)))
    print("Less than max_token_size!\n")
    summaries.append(art)
  else:
    chunks = chunkify(art)
    summary = summarize(chunks)
    summaries.append(summary)
    print("Added new summary!\n")

df = pd.DataFrame(summaries, columns=['summaries'])
df.to_csv("/content/khan_summaries.csv", index = False)

In [ ]:
df = pd.DataFrame(summaries, columns=['summaries'])
df.to_csv("/content/khan_summaries.csv", index = False)

Add some part of the original article to match the summary with the original

In [ ]:
matching = []
articles = article_unique.iloc[:,1]
articles = articles[:-1]
print(len(articles))
for art in articles:
  matching.append(art[:40])

1102


In [ ]:
summary_list = pd.DataFrame(list(zip(matching, list(summaries.iloc[:,0]))),
               columns =['match', 'summary'])

In [ ]:
summary_list.head()

,match,summary
0,pretty much as long as humans have looke,Three recent exoplanet discoveries are complet...
1,translator : jennifer cody reviewer : je,translator : jennifer cody reviewer : jessica ...
2,"vsauce , i ’ m jake and thank you for vi","vsauce, i’ m jake and thank you for visiting b..."
3,this episode of scishow might make you a,There have been thousands of viruses spread on...
4,"every day , a sea of decisions stretches","every day , a sea of decisions stretches befor..."


In [ ]:
summary_list.to_csv("summary_list.csv", index = False)